In [6]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from datetime import datetime
import time
import json
import psycopg2
import redis
import sys

In [7]:
reload(sys)
sys.setdefaultencoding('utf8') 

In [8]:
intent_table = pd.read_excel("INTENT_TAG.xlsx")
intent_table.head()

,TAG_ID,scenario,recommedWeight,modelId,operationType,dataSource,securityFilter,expireAfter,create Time,update Time,tagVersion,isActive,Chinese_Desc,referenceDocument
0,TG0001,I|LN|LN01,1,MINT00001,Batch,WB,admin,28,17385,17385,V1,Y,資金需求/無擔保/官網,INTENTION_REF
1,TG0002,I|LN|LN02,1,MINT00001,Batch,WB,admin,28,17385,17385,V1,Y,資金需求/房貸/官網,INTENTION_REF
2,TG0003,I|LN|LN03,1,MINT00001,Batch,WB,admin,28,17385,17385,V1,Y,資金需求/微型企業/官網,INTENTION_REF
3,TG0004,I|WM|WM01,1,MINT00001,Batch,WB,admin,28,17385,17385,V1,Y,投資理財/基金理財/官網,INTENTION_REF
4,TG0005,I|PL|PL01,1,MINT00001,Batch,WB,admin,28,17385,17385,V1,Y,享樂/百貨購物/官網,INTENTION_REF


In [9]:
intent_tag = list(intent_table)
intent_tag

[u'TAG_ID',
 u'scenario',
 u'recommedWeight',
 u'modelId',
 u'operationType',
 u'dataSource',
 u'securityFilter',
 u'expireAfter',
 u'create Time',
 u'update Time',
 u'tagVersion',
 u'isActive',
 u'Chinese_Desc',
 u'referenceDocument']

In [10]:
intent_table['intent_list']=''
intent_Json = []
for i in range(intent_table.shape[0]):
    temp_detail = {}
    for column in intent_tag:
        temp_detail[column]=str(intent_table.loc[i,:][column])
    
    scenario = intent_table.scenario[i].split('|')

    temp_detail['scenario'] = {'DIIS1' : '',
                               'DIIS2' : '',
                               'DIIS3' : ''}
    
    for j in range(len(scenario)):
        DIIS = 'DIIS' + str(j+1)
        temp_detail['scenario'][DIIS] = scenario[j]
    
    intent_Json.append(temp_detail)
print intent_Json

In [154]:
scenario

[u'S', u'SEG003', u'01']

In [155]:
temp_detail['scenario']

{'DIIS1': u'S', 'DIIS2': u'SEG003', 'DIIS3': u'01'}

In [141]:
"""
for row in range(len(intent_table)):
    scenario = intent_table.scenario[row].split('|')

    temp_detail['scenario'] = {'DIIS1' : '',
                               'DIIS2' : '',
                               'DIIS3' : ''}
    
    for i in range(len(scenario)):
        DIIS = 'DIIS' + str(i+1)
        temp_detail['scenario'][DIIS] = scenario[i]
"""

"\nfor row in range(len(intent_table)):\n    scenario = intent_table.scenario[row].split('|')\n\n    temp_detail['scenario'] = {'DIIS1' : '',\n                               'DIIS2' : '',\n                               'DIIS3' : ''}\n    \n    for i in range(len(scenario)):\n        DIIS = 'DIIS' + str(i+1)\n        temp_detail['scenario'][DIIS] = scenario[i]\n"

In [142]:
import psycopg2


# connect PostgreSQL
conn = psycopg2.connect(database = "MJ",
                        host = "localhost",
                        user = "llc",
                        password = "")
cur = conn.cursor()

In [144]:
for i in intent_Json:
    SQL = "INSERT INTO intent_db(tag_info) VALUES('{}' );".format(json.dumps(i))
    cur.execute(SQL)
    conn.commit()